In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
import requests
from urllib.parse import urlparse
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'
from time import sleep

In [2]:
# 지번주소로 도로명주소와 좌표를 다운로드함. 출처 - 행정안전부 도로명주소 안내시스템 개발자센터

def fetch_coor_data(df):
    df = df.copy()
    
    roadname_url = 'https://www.juso.go.kr/addrlink/addrLinkApi.do?'
    roadname_apikey = 'devU01TX0FVVEgyMDIxMDEyNDExMDM0MDExMDczNjE='
    coor_base_url = 'https://www.juso.go.kr/addrlink/addrCoordApi.do?'
    coor_apikey = 'devU01TX0FVVEgyMDIxMDIyNjExMDQwMDExMDg1ODU='
    # 도로명주소 검색 api key 만료일 2021년 4월 24일
    # 좌표검색 api key 만료일 2021년 5월 27일
    #apikey 만료시 https://www.juso.go.kr/addrlink/devAddrLinkRequestWrite.do?returnFn=write&cntcMenu=URL 가서 새로 받을 것.
    
    coor_data_df = pd.DataFrame({'지번주소':[], '도로명주소':[], '역명':[], 'x좌표':[], 'y좌표':[]})
    
    for i in tqdm(range(df.shape[0])):
        address_no_zero = df['역주소'].loc[i].split('(')[0]
        station_name = df['역명'].loc[i]
        if station_name.endswith('역') == False:
            station_name += '역'
            
        roadname_result = None
        coor_result = None
        coor_info = None
        
        roadname_result = requests.get(roadname_url + 'confmKey=' + roadname_apikey + '&currentPage=1' + '&countPerPage=5' + '&keyword=' \
                                       + address_no_zero + '&resultType=json' + '&hstryYn=Y')
        sleep(1)
        roadname_info = roadname_result.json()['results']['juso']
        
        the_idx = None        
        if roadname_info == []:
            roadname_result = requests.get(roadname_url + 'confmKey=' + roadname_apikey + '&currentPage=1' + '&countPerPage=5' + '&keyword=' \
                                       + station_name + '&resultType=json' + '&hstryYn=Y')
            sleep(1)
            roadname_info = roadname_result.json()['results']['juso']
            
        for j in range(len(roadname_info)):
            if roadname_info[j]['bdNm'] == station_name:
                the_idx = j
                break
        
        if the_idx == None:
            roadname_info = roadname_info[0]
        else:
            roadname_info = roadname_info[the_idx]
        
        coor_result = requests.get(coor_base_url + 'confmKey=' + coor_apikey + '&admCd=' + roadname_info['admCd']\
                                   + '&rnMgtSn=' + roadname_info['rnMgtSn'] + '&udrtYn=' + roadname_info['udrtYn']\
                                   + '&buldMnnm=' + roadname_info['buldMnnm'] + '&buldSlno=' + roadname_info['buldSlno']\
                                   + '&resultType=json')
        sleep(1)
        #return coor_result
        
        if coor_result.json()['results']['juso'] == None:
            print(address_no_zero)
            continue
        
        coor_info = coor_result.json()['results']['juso'][0]
        
        coor_x = pd.to_numeric(coor_info['entX'])
        coor_y = pd.to_numeric(coor_info['entY'])
        
        roadname_addr = roadname_info['roadAddrPart1']
        
        coor_data_df.loc[i,'지번주소'] = address_no_zero
        coor_data_df.loc[i,'도로명주소'] = roadname_addr
        coor_data_df.loc[i, '역명'] = station_name
        coor_data_df.loc[i, 'x좌표'] = coor_x
        coor_data_df.loc[i, 'y좌표'] = coor_y
        
    
    
    return coor_data_df

In [3]:
basedir = './지하철역/'
filenames = [f for f in os.listdir(basedir) if f.endswith('.xlsx')]
filenames

['서울교통공사_역주소 및 전화번호_20200715.xlsx']

In [4]:
subway_df = pd.read_excel(basedir + filenames[0])
print(subway_df.shape)
subway_df.head()

(292, 6)


,연번,역번호,호선,역명,역전화번호,역주소
0,1,150,1,서울역,02-6110-1331,서울특별시 중구 세종대로 지하 2 (남대문로 5가)
1,2,151,1,시청,02-6110-1321,서울특별시 중구 세종대로 지하 101 (정동)
2,3,152,1,종각,02-6110-1311,서울특별시 종로구 종로 지하 55 (종로1가)
3,4,153,1,종로3가,02-6110-1301,서울특별시 종로구 종로 지하 129 (종로3가)
4,5,154,1,종로5가,02-6110-1291,서울특별시 종로구 종로 지하 216 (종로5가)


In [5]:
subway_coor_df = fetch_coor_data(subway_df)

100%|████████████████████████████████████████████████████████████████████████████████| 292/292 [12:09<00:00,  2.50s/it]


In [6]:
print(subway_coor_df.shape)
subway_coor_df.head()

(292, 5)


,지번주소,도로명주소,역명,x좌표,y좌표
0,서울특별시 중구 세종대로 지하 2,서울특별시 중구 세종대로 지하 2,서울역,953414.452778,1.950998e+06
1,서울특별시 중구 세종대로 지하 101,서울특별시 중구 세종대로 지하 101,시청역,953808.786159,1.951915e+06
2,서울특별시 종로구 종로 지하 55,서울특별시 종로구 종로 지하 55,종각역,954366.404325,1.952442e+06
3,서울특별시 종로구 종로 지하 129,서울특별시 종로구 종로 지하 129,종로3가역,955141.774693,1.952461e+06
4,서울특별시 종로구 종로 지하 216,서울특별시 종로구 종로 지하 216,종로5가역,956016.193934,1.952509e+06


In [7]:
for v in subway_coor_df['역명'].unique():
    v_df = subway_coor_df[subway_coor_df['역명'] == v]
    
    if v_df.shape[0] > 1:
        for i in range(v_df.shape[0]):
            v_df.iloc[i, 2] = v + '_{}'.format(i)
        
        subway_coor_df.loc[v_df.index, '역명'] = v_df['역명']

In [8]:
subway_coor_df['역명'].nunique()

292

In [9]:
subway_coor_df['역명'].unique()

array(['서울역_0', '시청역_0', '종각역', '종로3가역_0', '종로5가역', '동대문역_0', '신설동역_0',
       '제기동역', '청량리역', '동묘앞역_0', '까치산역_0', '시청역_1', '을지로입구역', '을지로3가역_0',
       '을지로4가역_0', '동대문역사문화공원역_0', '신당역_0', '상왕십리역', '왕십리역_0', '한양대역',
       '뚝섬역', '성수역', '건대입구역_0', '구의역', '강변역', '잠실나루역', '잠실역_0', '잠실새내역',
       '종합운동장역_0', '삼성역', '선릉역', '역삼역', '강남역', '교대역_0', '서초역', '방배역',
       '사당역_0', '낙성대역', '서울대입구역', '봉천역', '신림역', '신대방역', '구로디지털단지역',
       '대림역_0', '신도림역', '문래역', '영등포구청역_0', '당산역', '합정역_0', '홍대입구역', '신촌역',
       '이대역', '아현역', '충정로역_0', '용답역', '신답역', '신설동역_1', '도림천역', '양천구청역',
       '신정네거리역', '용두역', '지축역', '구파발역', '연신내역_0', '불광역_0', '녹번역', '홍제역',
       '무악재역', '독립문역', '경복궁역', '안국역', '종로3가역_1', '을지로3가역_1', '충무로역_0',
       '동대입구역', '약수역_0', '금호역', '옥수역', '압구정역', '신사역', '잠원역', '고속터미널역_0',
       '교대역_1', '남부터미널역', '양재역', '매봉역', '도곡역', '대치역', '학여울역', '대청역',
       '일원역', '수서역', '가락시장역_0', '경찰병원역', '오금역_0', '당고개역', '상계역', '노원역_0',
       '창동역', '쌍문역', '수유역', '미아역', '미아사거리역', '길음역', '성신여대입구역', '한성

In [10]:
subway_coor_df.to_csv('./지하철역/subway_coor.csv', index=False)